In [3]:
!pip install -f https://download.pytorch.org/whl/cu101/torch_stable.html torch==1.8.0+cu101 torchvision==0.9.0
!pip install wandb==0.12.1

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu101/torch-1.8.0%2Bcu101-cp37-cp37m-linux_x86_64.whl (763.5 MB)
  Using cached https://download.pytorch.org/whl/cu101/torchvision-0.9.0%2Bcu101-cp37-cp37m-linux_x86_64.whl (17.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0
    Uninstalling torchvision-0.13.0:
      Successfully uninstalled torchvision-0.13.0


In [1]:
from tqdm import tqdm
from os import listdir
from random import *
seed(0)


import pandas as pd

from pylab import *

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torch.utils.data import DataLoader, random_split

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

import wandb

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cuda:0"

## Data trepanation

In [2]:
dataset = 'barents'
folder_train, folder_val, folder_test = dataset + '/train/maps/', dataset + '/valid/maps/', dataset + '/test/maps/'
train_files, val_files, test_files = sorted([file for file in listdir(folder_train)])[1346:],\
                                     sorted([file for file in listdir(folder_val)]),\
                                     sorted([file for file in listdir(folder_test)]) # + 1200

sample = torch.load(folder_train + train_files[1])
grid = torch.load(dataset + "/train/grid.pt")
coverage = pd.read_csv(dataset + "/train/coverage.csv")

len(train_files), len(val_files), len(test_files)

(1812, 366, 365)

In [3]:
keys = sample.keys()

In [4]:
def nan_check(tensor: torch.Tensor) -> bool:
    return True in tensor.isnan()

for key in keys:
    print(nan_check(sample[key]), key, sample[key].shape)

True cryosat.sic torch.Size([360, 500])
True cryosat.sit torch.Size([360, 500])
True jaxa.sic_asc torch.Size([360, 500])
True jaxa.sic_desc torch.Size([360, 500])
True jaxa.sic torch.Size([360, 500])
False gfs.f0d.temperature torch.Size([360, 500])
False gfs.f0d.pressure torch.Size([360, 500])
False gfs.f0d.wind_u torch.Size([360, 500])
False gfs.f0d.wind_v torch.Size([360, 500])
False gfs.f0d.wind torch.Size([360, 500])
False gfs.f1d.temperature torch.Size([360, 500])
False gfs.f1d.pressure torch.Size([360, 500])
False gfs.f1d.wind_u torch.Size([360, 500])
False gfs.f1d.wind_v torch.Size([360, 500])
False gfs.f1d.wind torch.Size([360, 500])
False gfs.f2d.temperature torch.Size([360, 500])
False gfs.f2d.pressure torch.Size([360, 500])
False gfs.f2d.wind_u torch.Size([360, 500])
False gfs.f2d.wind_v torch.Size([360, 500])
False gfs.f2d.wind torch.Size([360, 500])
False gfs.f3d.temperature torch.Size([360, 500])
False gfs.f3d.pressure torch.Size([360, 500])
False gfs.f3d.wind_u torch.Siz

In [5]:
def preprocess_image(tensor: torch.Tensor) -> torch.Tensor:
    return (torch.nan_to_num(tensor, nan=-10.0) + grid['land']*10)

In [6]:
gfs_keys = [item for item in sample.keys() if ('wind' in item and 'wind_' not in item) or 'temp' in item]

def handler(data: dict):
    try:
        return data["jaxa.sic"]
    except:
        return data["jaxa.sic_asc"]
    
def preprocess_image_gfs(tensor: torch.Tensor) -> torch.Tensor:
    layers = []
    
    for i in range(len(gfs_keys)):
        if gfs_keys[i] not in tensor.keys():
            tk = tensor.keys()
            print("Alarm")
            for key in tk:
                if key[-3:] == gfs_keys[i][-3:]:
                    layers.append(tensor[key])
                    break
        else:
            layers.append(tensor[gfs_keys[i]])
            
    sic_processed = torch.nan_to_num(handler(tensor), nan=-10.0) + grid['land']*10
    layers.append(sic_processed)
    
    return torch.stack(layers)
    
images_test_gfs = [preprocess_image_gfs(torch.load(folder_test + test_files[i]))\
                            for i in tqdm(range(0, len(test_files)))]

images_train_gfs = [preprocess_image_gfs(torch.load(folder_train + train_files[i]))\
                            for i in tqdm(range(0, len(train_files)))]

images_val_gfs = [preprocess_image_gfs(torch.load(folder_val + val_files[i]))\
                            for i in tqdm(range(0, len(val_files)))]

def average(data: list) -> float:
    return sum(data)/len(data)

def load_data_gfs(batch_size, images_tensor, d_in, d_out):
    in_batch, out_batch = [], []
    for i in range(batch_size):
        start_point = choice(range(len(images_tensor) - d_in - d_out))
        sic = images_tensor[start_point:start_point+d_in][:, -1, :, :]
        weather = images_tensor[start_point+d_in][:-1, :, :]
        in_batch.append(torch.cat((sic, weather), dim=0))
        out_batch.append(images_tensor[start_point+d_in:start_point+d_in+d_out][:, -1, :, :])
    return torch.stack(in_batch), torch.stack(out_batch)

 35%|███▌      | 638/1812 [00:04<00:08, 136.51it/s]

Alarm
Alarm


 57%|█████▋    | 1034/1812 [00:07<00:06, 119.04it/s]

Alarm
Alarm
Alarm
Alarm
Alarm
Alarm


 59%|█████▉    | 1071/1812 [00:08<00:06, 117.20it/s]

Alarm
Alarm
Alarm
Alarm


100%|██████████| 366/366 [00:02<00:00, 144.32it/s]


In [7]:
def train_gfs(model, criterion, d_in, d_out, epochs, batch_size, lr, eval_step, normalize):
    optimizer = optim.Adam(model.parameters(), lr=lr) #weight_decay=1e-8, momentum=0.9
#    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
#    grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)
    for epoch in range(1, epochs+1):
#        print("Current epoch", epoch)
        model.train()
        images_in, images_out = load_data_gfs(batch_size, images_tensor_train_gfs, d_in, d_out)
        optimizer.zero_grad()
        model_out = model(images_in[:, :, None, :, :])
        loss = criterion(model_out[1][0], images_out)
        loss.backward(retain_graph=True)
        optimizer.step()
        if epoch % eval_step == 0:
            mae_total, rmse_total, mape_total = [], [], []
            
            total_test_len = len(images_tensor_test_gfs)
            start_point, steps = 0, int(total_test_len/(d_in + d_out))
            
            for i in range(steps):
                sic = images_tensor_test_gfs[start_point:start_point+d_in][:, -1, :, :]
                weather = images_tensor_test_gfs[start_point+d_in][:-1, :, :]
                images_in = torch.cat((sic, weather), dim=0)
                
                images_out = images_tensor_test_gfs[start_point+d_in:start_point+d_in+d_out][:, -1, :, :]
                
                start_point += d_in + d_out
            
                model_out = model(images_in[None, :, None, :, :])
                
                criterion_mse = nn.MSELoss()
                
                loss_mse = criterion_mse(model_out[1][0], images_out[None, :, :, :])
                loss_mae = criterion(model_out[1][0], images_out[None, :, :, :])
                
                mae_total.append(loss_mae.detach().cpu().numpy())
                rmse_total.append(math.sqrt(loss_mse.detach().cpu().numpy()))
                
            
            wandb.log(
                {
                 "test/test_MAE": average(mae_total), 
                 "test/test_RMSE": average(rmse_total)
                }
            )
            
            print('Epoch ', epoch, ', test MAE - ', average(mae_total))

In [8]:
images_tensor_train_gfs, images_tensor_val_gfs, images_tensor_test_gfs = torch.stack(images_train_gfs).to(device),\
                                                             torch.stack(images_val_gfs).to(device),\
                                                             torch.stack(images_test_gfs).to(device)

In [9]:
architecture = "ConvNet_GFS_7:3"
i = 0
wandb.init(project="SeaIcePrediction", entity="eighonet", group=architecture)
wandb.run.name = architecture + "_" + str(i)  # "gs_3_ffd_3_128_test_MAEloss_lr_10^{-4}_10000"
wandb.run.save()

wandb: Currently logged in as: sbercv (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-07-26 15:27:50.605507: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [11]:
from models.convlstm.convlstm import ConvLSTM

epochs = 1000
d_in, d_out = 7, 3
batch_size = 1
lr = 1e-2
eval_step = 20
criterion = nn.L1Loss()
model_a = ConvLSTM(input_dim=1,
                 hidden_dim=[32, 32, 32],
                 kernel_size=(3, 3),
                 num_layers=3,
                 batch_first=True,
                 bias=True,
                 return_all_layers=False,
                 final_layer=True,
                 num_days=3).to(device)

In [12]:
47 -33

14

In [13]:
input_data = torch.zeros(2, 7, 15, 360, 500).to(device)

In [14]:
model_a(input_data)[1][0].shape

RuntimeError: Given groups=1, weight of size [128, 33, 3, 3], expected input[2, 47, 360, 500] to have 33 channels, but got 47 channels instead

In [15]:
train_gfs(model_a, criterion, d_in, d_out, epochs, batch_size, lr, eval_step, "False")

RuntimeError: CUDA out of memory. Tried to allocate 22.00 MiB (GPU 0; 31.72 GiB total capacity; 29.77 GiB already allocated; 8.44 MiB free; 30.39 GiB reserved in total by PyTorch)

## Model trepanation

In [7]:
images_train = [preprocess_image(torch.load(folder_train + train_files[i])["jaxa.sic"])\
                            for i in tqdm(range(0, len(train_files)))]

images_val = [preprocess_image(torch.load(folder_val + val_files[i])["jaxa.sic"])\
                            for i in tqdm(range(0, len(val_files)))]

def handler(data: dict):
    try:
        return data["jaxa.sic"]
    except:
        return data["jaxa.sic_asc"]
    
images_test = [preprocess_image(handler(torch.load(folder_test + test_files[i])))\
                            for i in tqdm(range(0, len(test_files)))]

 14%|█▍        | 259/1812 [00:04<00:28, 55.20it/s]ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/user/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_9041/2069040888.py", line 2, in <module>
    for i in tqdm(range(0, len(train_files)))]
  File "/tmp/ipykernel_9041/2069040888.py", line 2, in <listcomp>
    for i in tqdm(range(0, len(train_files)))]
  File "/home/jovyan/.imgenv-bert-large-0/lib/python3.7/site-packages/torch/serialization.py", line 579, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/home/jovyan/.imgenv-bert-large-0/lib/python3.7/site-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/home/jovyan/.imgenv-bert-large-0/lib/python3.7/site-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
KeyboardInterrupt

During handling of the above exception, another except

TypeError: object of type 'NoneType' has no len()

In [7]:
images_tensor_train, images_tensor_val, images_tensor_test = torch.stack(images_train).to(device),\
                                                             torch.stack(images_val).to(device),\
                                                             torch.stack(images_test).to(device)
images_tensor_train.shape

torch.Size([1812, 360, 500])

In [8]:
%load_ext autoreload
%autoreload 2

from models.convlstm.convlstm import ConvLSTM

In [9]:
def average(data: list) -> float:
    return sum(data)/len(data)

def load_data(batch_size, images_tensor, d_in, d_out):
    in_batch, out_batch = [], []
    for i in range(batch_size):
        start_point = choice(range(len(images_tensor) - d_in - d_out))
        in_batch.append(images_tensor[start_point:start_point+d_in])
        out_batch.append(images_tensor[start_point+d_in:start_point+d_in+d_out])
    return torch.stack(in_batch), torch.stack(out_batch)

def train(model, criterion, d_in, d_out, epochs, batch_size, lr, eval_step):
    optimizer = optim.Adam(model.parameters(), lr=lr) #weight_decay=1e-8, momentum=0.9
#    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
#    grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)
    for epoch in range(1, epochs+1):
#        print("Current epoch", epoch)
        model.train()
        images_in, images_out = load_data(batch_size, images_tensor_train, d_in, d_out)
        optimizer.zero_grad()
        model_out = model(images_in[:, :, None, :, :])
        loss = criterion(model_out[1][0], images_out)
        loss.backward(retain_graph=True)
        optimizer.step()
        if epoch % eval_step == 0:
            mae_total, rmse_total, mape_total = [], [], []
            
            total_test_len = len(images_tensor_test)
            start_point, steps = 0, int(total_test_len/(d_in + d_out))
            
            for i in range(steps):
                images_in, images_out = images_tensor_test[start_point:start_point+d_in],\
                                        images_tensor_test[start_point+d_in:start_point+d_in+d_out]
                
                start_point += d_in + d_out
            
                model_out = model(images_in[None, :, None, :, :])
                
                criterion_mse = nn.MSELoss()
                
                loss_mse = criterion_mse(model_out[1][0], images_out[None, :, :, :])
                loss_mae = criterion(model_out[1][0], images_out[None, :, :, :])
                
                mae_total.append(loss_mae.detach().cpu().numpy())
                rmse_total.append(math.sqrt(loss_mse.detach().cpu().numpy()))
                
            
            wandb.log(
                {
                 "test/test_MAE": average(mae_total), 
                 "test/test_RMSE": average(rmse_total)
                }
            )
            
            print('Epoch ', epoch, ', test MAE - ', average(mae_total))

In [10]:
architecture = "LSTMConv_7:10"
i = 0
wandb.init(project="SeaIcePrediction", entity="eighonet", group=architecture)
wandb.run.name = architecture + "_" + str(i)  # "gs_3_ffd_3_128_test_MAEloss_lr_10^{-4}_10000"
wandb.run.save()

wandb: Currently logged in as: sbercv (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-07-26 14:00:39.596283: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [11]:
epochs = 1000
d_in, d_out = 7, 10
batch_size = 2
lr = 1e-2
eval_step = 20
criterion = nn.L1Loss()
model_a = ConvLSTM(input_dim=1,
                 hidden_dim=[32, 32, 32],
                 kernel_size=(3, 3),
                 num_layers=3,
                 batch_first=True,
                 bias=True,
                 return_all_layers=False,
                 final_layer=True,
                 num_days=10).to(device)

train(model_a, criterion, d_in, d_out, epochs, batch_size, lr, eval_step)

Epoch  20 , test MAE -  9.299148559570312
Epoch  40 , test MAE -  5.907796223958333
Epoch  60 , test MAE -  4.075190952845982
Epoch  80 , test MAE -  3.458926972888765
Epoch  100 , test MAE -  3.531194051106771
Epoch  120 , test MAE -  3.486724853515625
Epoch  140 , test MAE -  3.390653337751116
Epoch  160 , test MAE -  3.267154511951265
Epoch  180 , test MAE -  3.3437917800176713
Epoch  200 , test MAE -  3.2690702165876115
Epoch  220 , test MAE -  3.1048551286969865
Epoch  240 , test MAE -  3.27815428234282
Epoch  260 , test MAE -  3.2051823933919272
Epoch  280 , test MAE -  3.715432666596912
Epoch  300 , test MAE -  3.2678389776320684
Epoch  320 , test MAE -  2.859410603841146
Epoch  340 , test MAE -  2.9050599961053756
Epoch  360 , test MAE -  2.856763930547805
Epoch  380 , test MAE -  2.961596352713449
Epoch  400 , test MAE -  2.940976824079241
Epoch  420 , test MAE -  2.9703521728515625
Epoch  440 , test MAE -  2.8600253150576638
Epoch  460 , test MAE -  2.8381073361351374
Epoch  

In [12]:
train(model_a, criterion, d_in, d_out, epochs, batch_size, 1e-3, eval_step)

Epoch  20 , test MAE -  2.7056739443824407
Epoch  40 , test MAE -  2.683204832531157
Epoch  60 , test MAE -  2.6731089637393044
Epoch  80 , test MAE -  2.669137500581287
Epoch  100 , test MAE -  2.6592276436941966
Epoch  120 , test MAE -  2.6481830051967075
Epoch  140 , test MAE -  2.64355959211077
Epoch  160 , test MAE -  2.6396611531575522
Epoch  180 , test MAE -  2.649632771809896
Epoch  200 , test MAE -  2.641253698439825
Epoch  220 , test MAE -  2.621009281703404
Epoch  240 , test MAE -  2.656551724388486
Epoch  260 , test MAE -  2.662096477690197
Epoch  280 , test MAE -  2.650421142578125
Epoch  300 , test MAE -  2.6257945469447543
Epoch  320 , test MAE -  2.6294987996419272
Epoch  340 , test MAE -  2.6449993678501675
Epoch  360 , test MAE -  2.610768817719959
Epoch  380 , test MAE -  2.6061072576613653
Epoch  400 , test MAE -  2.663219996861049
Epoch  420 , test MAE -  2.628437042236328
Epoch  440 , test MAE -  2.6122064136323475
Epoch  460 , test MAE -  2.6047920953659784
Epoch

In [13]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [115]:
%debug

> /home/jovyan/Ice/models/convlstm/convlstm.py(140)forward()
    138             input_tensor = input_tensor.permute(1, 0, 2, 3, 4)
    139 
--> 140         b, _, _, h, w = input_tensor.size()
    141 
    142         # Implement stateful ConvLSTM

ipdb> images_in.shape
*** NameError: name 'images_in' is not defined
ipdb> input_tensor.shape
torch.Size([8, 7, 360, 500])
ipdb> input_tensor[:, :, None, :, :].shape
torch.Size([8, 7, 1, 360, 500])
ipdb> exit


In [17]:
input_data = torch.zeros(2, 7, 1, 360, 500).to(device)

In [18]:
model_output = model_a(input_data)

RuntimeError: CUDA out of memory. Tried to allocate 352.00 MiB (GPU 0; 31.72 GiB total capacity; 28.99 GiB already allocated; 144.44 MiB free; 30.26 GiB reserved in total by PyTorch)

In [108]:
len(model_output[0])

1

In [109]:
model_output[0][0].shape

torch.Size([2, 7, 3, 360, 500])

In [110]:
model_output[1][0].shape

torch.Size([2, 3, 360, 500])

In [77]:
conv1 = nn.Conv2d(128, 3, kernel_size=(3, 3), padding=(1, 1), bias=False)

In [78]:
conv1(model_output[1][0][0]).shape

torch.Size([2, 3, 260, 500])

In [37]:
model.cell_list[0]

ConvLSTMCell(
  (conv): Conv2d(65, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [54]:
architecture = "UNet_default"
i = 0
wandb.init(project="SeaIcePrediction", entity="eighonet", group=architecture)
wandb.run.name = architecture + "_" + str(i)  # "gs_3_ffd_3_128_test_MAEloss_lr_10^{-4}_10000"
wandb.run.save()

wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-07-24 15:10:14.801175: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [56]:
from models.unet.unet.unet_model import UNet

epochs = 1000
d_in, d_out = 7, 3
batch_size = 8
lr = 1e-3
eval_step = 20
criterion = nn.L1Loss()
model_a = UNet(7, 3).to(device)

train(model_a, criterion, d_in, d_out, epochs, batch_size, lr, eval_step)

Epoch  20 , test MAE -  12.035553826226128
Epoch  40 , test MAE -  11.641056484646267
Epoch  60 , test MAE -  11.30771213107639
Epoch  80 , test MAE -  11.037241617838541
Epoch  100 , test MAE -  10.864744398328993
Epoch  120 , test MAE -  10.708236694335938
Epoch  140 , test MAE -  10.546531677246094
Epoch  160 , test MAE -  10.378957960340712
Epoch  180 , test MAE -  10.214975992838541
Epoch  200 , test MAE -  10.031992594401041
Epoch  220 , test MAE -  9.869924757215712
Epoch  240 , test MAE -  9.698144700792101
Epoch  260 , test MAE -  9.531196594238281
Epoch  280 , test MAE -  9.329873826768663
Epoch  300 , test MAE -  9.15167490641276
Epoch  320 , test MAE -  8.956125895182291
Epoch  340 , test MAE -  8.773064507378471
Epoch  360 , test MAE -  8.569566514756945
Epoch  380 , test MAE -  8.382956610785591
Epoch  400 , test MAE -  8.162799411349827
Epoch  420 , test MAE -  7.9627634684244795
Epoch  440 , test MAE -  7.747506883409288
Epoch  460 , test MAE -  7.530576917860243
Epoch 

## UNet GFS

In [5]:
gfs_keys = [item for item in sample.keys() if ('wind' in item and 'wind_' not in item) or 'temp' in item]
gfs_keys

['gfs.f0d.temperature',
 'gfs.f0d.wind',
 'gfs.f1d.temperature',
 'gfs.f1d.wind',
 'gfs.f2d.temperature',
 'gfs.f2d.wind',
 'gfs.f3d.temperature',
 'gfs.f3d.wind']

In [6]:
def handler(data: dict):
    try:
        return data["jaxa.sic"]
    except:
        return data["jaxa.sic_asc"]
    
def preprocess_image_gfs(tensor: torch.Tensor) -> torch.Tensor:
    layers = []
    
    for i in range(len(gfs_keys)):
        if gfs_keys[i] not in tensor.keys():
            tk = tensor.keys()
            print("Alarm")
            for key in tk:
                if key[-3:] == gfs_keys[i][-3:]:
                    layers.append(tensor[key])
                    break
        else:
            layers.append(tensor[gfs_keys[i]])
            
    sic_processed = torch.nan_to_num(handler(tensor), nan=-10.0) + grid['land']*10
    layers.append(sic_processed)
    
    return torch.stack(layers)
    
images_test_gfs = [preprocess_image_gfs(torch.load(folder_test + test_files[i]))\
                            for i in tqdm(range(0, len(test_files)))]

images_train_gfs = [preprocess_image_gfs(torch.load(folder_train + train_files[i]))\
                            for i in tqdm(range(0, len(train_files)))]

images_val_gfs = [preprocess_image_gfs(torch.load(folder_val + val_files[i]))\
                            for i in tqdm(range(0, len(val_files)))]


 35%|███▌      | 639/1812 [00:05<00:11, 103.20it/s]

Alarm
Alarm


 57%|█████▋    | 1029/1812 [00:09<00:07, 98.45it/s] 

Alarm
Alarm
Alarm
Alarm
Alarm
Alarm


 58%|█████▊    | 1059/1812 [00:10<00:07, 94.73it/s]

Alarm
Alarm
Alarm
Alarm


100%|██████████| 366/366 [00:03<00:00, 93.70it/s] 


In [7]:
images_tensor_train_gfs, images_tensor_val_gfs, images_tensor_test_gfs = torch.stack(images_train_gfs).to(device),\
                                                             torch.stack(images_val_gfs).to(device),\
                                                             torch.stack(images_test_gfs).to(device)
images_tensor_train_gfs.shape

torch.Size([1812, 9, 360, 500])

In [8]:
def normalize(images_tensor_train_gfs):
    data = images_tensor_train_gfs.cpu().numpy()
    data_min = np.min(data, axis=(1,2,3), keepdims=True)
    data_max = np.max(data, axis=(1,2,3), keepdims=True)

    scaled_data = (data - data_min) / (data_max - data_min)
    return torch.Tensor(scaled_data)

In [ ]:
images_tensor_train_gfs, images_tensor_val_gfs, images_tensor_test_gfs = normalize(images_tensor_train_gfs).to(device),\
normalize(images_tensor_val_gfs).to(device),\
normalize(images_tensor_test_gfs).to(device),\

In [83]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
architecture = "UNet_GFS_norm"
i = 0
wandb.init(project="SeaIcePrediction", entity="eighonet", group=architecture)
wandb.run.name = architecture + "_" + str(i)  # "gs_3_ffd_3_128_test_MAEloss_lr_10^{-4}_10000"
wandb.run.save()

In [ ]:
def average(data: list) -> float:
    return sum(data)/len(data)

def load_data_gfs(batch_size, images_tensor, d_in, d_out):
    in_batch, out_batch = [], []
    for i in range(batch_size):
        start_point = choice(range(len(images_tensor) - d_in - d_out))
        sic = images_tensor[start_point:start_point+d_in][:, -1, :, :]
        weather = images_tensor[start_point+d_in][:-1, :, :]
        in_batch.append(torch.cat((sic, weather), dim=0))
        out_batch.append(images_tensor[start_point+d_in:start_point+d_in+d_out][:, -1, :, :])
    return torch.stack(in_batch), torch.stack(out_batch)

def train_gfs(model, criterion, d_in, d_out, epochs, batch_size, lr, eval_step, normalize):
    optimizer = optim.Adam(model.parameters(), lr=lr) #weight_decay=1e-8, momentum=0.9
#    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
#    grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)
    for epoch in range(1, epochs+1):
#        print("Current epoch", epoch)
        model.train()
        images_in, images_out = load_data_gfs(batch_size, images_tensor_train_gfs, d_in, d_out)
        optimizer.zero_grad()
        model_out = model(images_in)
        loss = criterion(model_out, images_out)
        loss.backward(retain_graph=True)
        optimizer.step()
        if epoch % eval_step == 0:
            mae_total, rmse_total, mape_total = [], [], []
            
            total_test_len = len(images_tensor_test_gfs)
            start_point, steps = 0, int(total_test_len/(d_in + d_out))
            
            for i in range(steps):
                sic = images_tensor_test_gfs[start_point:start_point+d_in][:, -1, :, :]
                weather = images_tensor_test_gfs[start_point+d_in][:-1, :, :]
                images_in = torch.cat((sic, weather), dim=0)
                
                images_out = images_tensor_test_gfs[start_point+d_in:start_point+d_in+d_out][:, -1, :, :]
                
                start_point += d_in + d_out
            
                model_out = model(images_in[None, :, :, :])
                
                criterion_mse = nn.MSELoss()
                
                loss_mse = criterion_mse(model_out, images_out[None, :, :, :])
                loss_mae = criterion(model_out, images_out[None, :, :, :])
                
                mae_total.append(loss_mae.detach().cpu().numpy())
                rmse_total.append(math.sqrt(loss_mse.detach().cpu().numpy()))
                
            
            wandb.log(
                {
                 "test/test_MAE": average(mae_total), 
                 "test/test_RMSE": average(rmse_total)
                }
            )
            
            print('Epoch ', epoch, ', test MAE - ', average(mae_total))

In [45]:
from models.unet.unet.unet_model import UNet

epochs = 1000
d_in, d_out = 7, 3
batch_size = 2
lr = 1e-3
eval_step = 20
normalize = True
criterion = nn.L1Loss()
model_a = UNet(15, 3).to(device)

train_gfs(model_a, criterion, d_in, d_out, epochs, batch_size, lr, eval_step, normalize)

Error: You must call wandb.init() before wandb.log()

In [76]:
def normalize(images_tensor_train_gfs_np):
    data = images_tensor_train_gfs.cpu().numpy()
    data_min = np.min(data, axis=(1,2,3), keepdims=True)
    data_max = np.max(data, axis=(1,2,3), keepdims=True)

    scaled_data = (data - data_min) / (data_max - data_min)
    return torch.Tensor(scaled_data)

In [61]:
images_tensor_train_gfs_np = images_tensor_train_gfs.cpu().numpy()

In [62]:
data = images_tensor_train_gfs_np

In [70]:
import numpy as np

# data = np.random.normal(loc=0, scale=1, size=(96108, 7, 7))
data_min = np.min(data, axis=(1,2,3), keepdims=True)
data_max = np.max(data, axis=(1,2,3), keepdims=True)

scaled_data = (data - data_min) / (data_max - data_min)

In [71]:
scaled_data.shape

(1812, 9, 360, 500)

In [72]:
scaled_data[0][0]

array([[0.9777034 , 0.9773288 , 0.9768093 , ..., 0.96315116, 0.96308655,
        0.9630696 ],
       [0.977667  , 0.9772811 , 0.97686255, ..., 0.96296054, 0.9629497 ,
        0.96293   ],
       [0.9776438 , 0.97724694, 0.97685874, ..., 0.96282965, 0.9628245 ,
        0.96276325],
       ...,
       [0.9847687 , 0.98451895, 0.98426574, ..., 0.92624736, 0.9258093 ,
        0.92527354],
       [0.98462695, 0.98437583, 0.9841222 , ..., 0.92687666, 0.92632633,
        0.92608243],
       [0.98448104, 0.98422205, 0.98397183, ..., 0.92726815, 0.927024  ,
        0.9266455 ]], dtype=float32)

In [73]:
scaled_data[0][-1]

array([[0.03487178, 0.03487178, 0.03487178, ..., 0.03487178, 0.03487178,
        0.03487178],
       [0.03487178, 0.03487178, 0.03487178, ..., 0.03487178, 0.03487178,
        0.03487178],
       [0.03487178, 0.03487178, 0.03487178, ..., 0.03487178, 0.03487178,
        0.03487178],
       ...,
       [0.03487178, 0.03487178, 0.03487178, ..., 0.03487178, 0.03487178,
        0.03487178],
       [0.03487178, 0.03487178, 0.03487178, ..., 0.03487178, 0.03487178,
        0.03487178],
       [0.03487178, 0.03487178, 0.03487178, ..., 0.03487178, 0.03487178,
        0.03487178]], dtype=float32)

In [75]:
images_tensor_train_gfs[0][0]

tensor([[270.3709, 270.2635, 270.1146,  ..., 266.1979, 266.1794, 266.1745],
        [270.3605, 270.2498, 270.1298,  ..., 266.1432, 266.1401, 266.1345],
        [270.3539, 270.2401, 270.1287,  ..., 266.1057, 266.1042, 266.0866],
        ...,
        [272.3970, 272.3254, 272.2528,  ..., 255.6152, 255.4896, 255.3359],
        [272.3564, 272.2844, 272.2116,  ..., 255.7956, 255.6378, 255.5679],
        [272.3145, 272.2403, 272.1685,  ..., 255.9079, 255.8379, 255.7293]],
       device='cuda:1')

In [46]:
t = torch.tensor([[1.,2.,3.],[4.,5.,6.]])
print("Tensor:", t)

Tensor: tensor([[1., 2., 3.],
        [4., 5., 6.]])


In [48]:
t1 = normalize(t, p=1.0, dim = 1)
t2 = normalize(t, p=2.0, dim = 0)

TypeError: 'bool' object is not callable

In [47]:
torch.unique(images_tensor_train_gfs[0][-1])

tensor([-1.0000e+01,  0.0000e+00,  3.1983e-05,  ...,  1.0000e+02,
         1.0000e+02,  1.0000e+02], device='cuda:1')

In [27]:
from torch.nn.functional import normalize


In [ ]:

normalize_layer = torch.nn.functional.normalize(input, p=2.0, dim=1, eps=1e-12, out=None)

### Prebuild

In [29]:
%load_ext autoreload
%autoreload 2

from models.unet3d.unet3d.unet3d.model import UNet3D, ResidualUNet3D

In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
model_3d = UNet3D(7, 3).to(device)
model_3d_r = ResidualUNet3D(7, 3).to(device)

In [16]:
init = images_tensor_train_gfs[:7][None, :, :, :, :]
out = F.pad(init, (0, 0, 0, 0, 2, 2), "constant", 0)
init.shape, out.shape

(torch.Size([1, 7, 6, 360, 500]), torch.Size([1, 7, 10, 360, 500]))

In [34]:
criterion = nn.MSELoss()

In [35]:
criterion(model_3d(out).squeeze(), images_tensor_test[0+7:0+7+3])

It works!


tensor(863.4682, device='cuda:0', grad_fn=<MseLossBackward>)

In [30]:
model_3d(out).squeeze().shape

It works!


torch.Size([3, 360, 500])

In [21]:
load_data(2, images_tensor_train_gfs, 7, 3)[1].shape

torch.Size([2, 3, 6, 360, 500])

In [28]:
images_tensor_test_gfs[0:0+7][None, :, :, :, :].shape, images_tensor_test_gfs[0+7:0+7+3].shape

(torch.Size([1, 7, 6, 360, 500]), torch.Size([3, 6, 360, 500]))

In [26]:
images_tensor_test_gfs[start_point+d_in:start_point+d_in+d_out].shape

NameError: name 'start_point' is not defined

In [10]:
def load_data_3d(batch_size, images_tensor, d_in, d_out):
    in_batch, out_batch = [], []
    for i in range(batch_size):
        start_point = choice(range(len(images_tensor) - d_in - d_out))
        in_batch.append(images_tensor[start_point:start_point+d_in])
        out_batch.append(images_tensor[start_point+d_in:start_point+d_in+d_out])
    return torch.stack(in_batch), torch.stack(out_batch)


def train_3d(model, criterion, d_in, d_out, epochs, batch_size, lr, eval_step):
    optimizer = optim.Adam(model.parameters(), lr=lr) #weight_decay=1e-8, momentum=0.9
#    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
#    grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)
    for epoch in range(1, epochs+1):
        print("Current epoch", epoch)
        model.train()
        images_in, images_out = load_data_3d(batch_size, images_tensor_train_gfs, d_in, d_out)
        optimizer.zero_grad()
        images_in_padded = F.pad(images_in, (0, 0, 0, 0, 1, 1), "constant", 0)
        model_out = model(images_in_padded)
        loss = criterion(model_out, images_out[:, :, -1, :, :])
        loss.backward(retain_graph=True)
        optimizer.step()
        if epoch % eval_step == 0:
            mae_total, rmse_total, mape_total = [], [], []
            
            total_test_len = len(images_tensor_test_gfs)
            start_point, steps = 0, int(total_test_len/(d_in + d_out))
            
            for i in tqdm(range(steps)):
                images_in, images_out = images_tensor_test_gfs[start_point:start_point+d_in],\
                                        images_tensor_test_gfs[start_point+d_in:start_point+d_in+d_out][:, -1, :, :]
                
                start_point += d_in + d_out
                
                images_in_padded = F.pad(images_in, (0, 0, 0, 0, 1, 1), "constant", 0)
                model_out = model(images_in_padded[None, :, :, :, :]).squeeze()
                
                criterion_mse = nn.MSELoss()
                
                loss_mse = criterion_mse(model_out, images_out[None, :, :, :])
                loss_mae = criterion(model_out, images_out[None, :, :, :])
                
                mae_total.append(loss_mae.detach().cpu().numpy())
                rmse_total.append(math.sqrt(loss_mse.detach().cpu().numpy()))
                del images_in, images_out, model_out
                gc.collect()
                torch.cuda.empty_cache()
            
            wandb.log(
                {
                 "test/test_MAE": average(mae_total), 
                 "test/test_RMSE": average(rmse_total)
                }
            )
            
            print('Epoch ', epoch, ', test MAE - ', average(mae_total))

In [11]:
architecture = "UNet3D_default"
i = 0
wandb.init(project="SeaIcePrediction", entity="eighonet", group=architecture)
wandb.run.name = architecture + "_" + str(i)  # "gs_3_ffd_3_128_test_MAEloss_lr_10^{-4}_10000"
wandb.run.save()

wandb: Currently logged in as: sbercv (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-07-24 11:31:50.786818: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [14]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [15]:
%load_ext autoreload
%autoreload 2
from models.unet3d.unet3d.unet3d.model import UNet3D, ResidualUNet3D

epochs = 100
d_in, d_out = 7, 3
batch_size = 1
lr = 1e-3
eval_step = 10
criterion = nn.L1Loss()
model_3d = UNet3D(7, 3).to(device)

train_3d(model_3d, criterion, d_in, d_out, epochs, batch_size, lr, eval_step)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current epoch 1


RuntimeError: CUDA out of memory. Tried to allocate 1.03 GiB (GPU 0; 31.72 GiB total capacity; 29.64 GiB already allocated; 746.44 MiB free; 29.67 GiB reserved in total by PyTorch)

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Cl

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Cl

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Cl

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Cl

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Cl

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Cl

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/eighonet/SeaIcePrediction/1t5cu1nc/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Cl

In [1]:
UNet3D(7, 3)

NameError: name 'UNet3D' is not defined

### Manual

In [41]:
out_channels = 3
test_conv3d = nn.Conv3d(7, 1, kernel_size=3, padding=1, bias=False).to(device)
test_conv2d = nn.Conv2d(6, out_channels, kernel_size=3, padding=1, bias=False).to(device)

In [42]:
test_conv3d(images_tensor_train_gfs[:7][None, :, :, :, :]).squeeze(1).shape
result_prj = test_conv3d(images_tensor_train_gfs[:7][None, :, :, :, :]).squeeze(1)
test_conv2d(result_prj).shape

torch.Size([1, 3, 360, 500])

In [39]:
images_tensor_train_gfs[:7][None, :, :, :, :].shape

torch.Size([1, 7, 6, 360, 500])

In [40]:
result_prj.shape

torch.Size([1, 6, 360, 500])

In [14]:
%load_ext autoreload
%autoreload 2

In [15]:
%load_ext autoreload
%autoreload 2

from models.unet.unet.unet_model_3d import UNet_3d

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
model_3d = UNet_3d(7, 3).to(device)

In [17]:
torch.zeros([1, 256, 1, 90, 125]).squeeze(2).shape

torch.Size([1, 256, 90, 125])

In [18]:
model_3d

UNet_3d(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv3d(7, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 

In [19]:
model_3d(images_tensor_train_gfs[:7][None, :, :, :, :])

torch.Size([1, 128, 3, 180, 250])
torch.Size([1, 256, 90, 125])
torch.Size([1, 512, 45, 62])
torch.Size([1, 1024, 22, 31])


RuntimeError: Tensors must have same number of dimensions: got 4 and 5

In [20]:
%debug

> /home/jovyan/Ice/models/unet/unet/unet_parts_3d.py(99)forward()
     97         # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
     98         # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
---> 99         x = torch.cat([x2, x1], dim=1)
    100         return self.conv(x)
    101 

ipdb> x2.shape
torch.Size([1, 128, 3, 180, 250])
ipdb> x1.shape
torch.Size([1, 128, 3, 180])
ipdb> exit
